# Structuration DataSet (A ne plus relancer)

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# On concaténe les 6 fichiers
import pandas as pd
df_0 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_0.csv")
df_1 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_3.csv")
df_4 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_4.csv")
df_5 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_5.csv")
# df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron.csv", nrows=1500)
frames = [df_0,df_1,df_2,df_3,df_4,df_5]
df = pd.concat(frames)
# la selection ci-dessous a permis de détecter 3 doublons (toutes colonnes identiques)
# df = df.loc[(df['subject'] == "Sagewood Town Homes")]
print(df.shape)
# df = df_emails.loc[(df['subject'] == "Sagewood Town Homes")]
df.drop('Unnamed: 0', axis=1, inplace=True)
# recherche
# suppression des doublons (sur toutes les colonnes sauf index)
df.drop_duplicates(subset=['date','sender', 'recipient1', 'subject','text'],keep = 'first', inplace=True)
# on renomme les colonnes
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})
# et export en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv"
df.to_csv(file_name, encoding='utf-8', index=False)
print(df.shape)
# Taille du fichier divisée par 2 environ
# génération d'un fichier light de 1000 lignes pour traitements plus rapides.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_1000.csv"
df_emails.to_csv(file_name, encoding='utf-8', index=False)

Mounted at /content/drive
(405968, 6)
(227032, 5)


# Import - Data Preprocessing structures

Mount Gdrive files

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import Spacy

In [ ]:
#!pip install -U spacy
!python -m spacy download en_core_web_lg
# ligne ci-dessous a réexecuter ssi reinit du notebook
!pip install spacy-transformers

2022-08-01 13:13:28.380644: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 8.2 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 27.4 MB/s 
     |████████████████████████████████| 4.4 MB 55.2 MB/s 
     |████████████████████████████████| 6.6 MB 43.8 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstall

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'


In [ ]:
import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df_emails = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_1000.csv",nrows=500)
# , skiprows=1
df_emails.tail(10)

,date,from,to,header,body
490,2001-01-24 23:40:00-08:00,phillip.allen@enron.com,stagecoachmama@hotmail.com,NaN,"['', 'Lucy,', '', 'Here is a rentroll for this..."
491,2001-01-24 09:23:00-08:00,phillip.allen@enron.com,llewter@austin.rr.com,NaN,"['X-Origin: Allen-P', 'X-FileName: pallen.nsf'..."
492,2001-01-22 06:06:00-08:00,phillip.allen@enron.com,john.lavorato@enron.com,Re: Response to PGE request for gas,"['', '---------------------- Forwarded by Phil..."
493,2001-01-22 01:24:00-08:00,phillip.allen@enron.com,mike.grigsby@enron.com,NaN,"['', 'By STEVE EVERLY - The Kansas City Star',..."
494,2001-01-22 00:38:00-08:00,phillip.allen@enron.com,ina.rangel@enron.com,Re: Please respond,"['', 'message board']"
495,2001-01-22 00:34:00-08:00,phillip.allen@enron.com,ina.rangel@enron.com,Re: Please respond,"['', 'need help. ']"
496,2000-12-06 04:41:00-08:00,phillip.allen@enron.com,del@living.com,Re: Court Ordered Notice to Customers and Regi...,"['X-FileName: pallen.nsf', '', 'please remove ..."
497,2001-01-18 02:12:00-08:00,phillip.allen@enron.com,jsmith@austintx.com,NaN,"['', 'Jeff,', '', 'Here is a recent rentroll. ..."
498,2001-01-18 02:08:00-08:00,phillip.allen@enron.com,llewter@austin.rr.com,NaN,"['', 'Larry,', '', 'The wire should go out tod..."
499,2001-01-17 07:35:00-08:00,phillip.allen@enron.com,stagecoachmama@hotmail.com,NaN,"['', 'Lucy,', '', 'Why did so many tenants not..."


Rajout colonnes pour traitements ultérieurs

In [ ]:
import numpy as np
df_emails = df_emails.replace('NaN', np.NaN)
# df_emails = df[df['header'].notna()]

df_emails['cc'] = ''                        # pour identfication des personnes en copie
df_emails['NER_header'] = ''                # Spacy NER
df_emails['NER_body'] = ''                  # Spacy NER
df_emails['body_clean'] = ''                # body sans caracteres parasites
df_emails['body_dict'] = ''                 # body tokenisé
df_emails['summary_TFIDF'] = ''             # extractive summary by TF-IDF
df_emails['summary_TFIDF_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_spacy'] = ''             # extractive summary by Spacy
df_emails['summary_spacy_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_BART'] = ''              # abstractive summary by HugginFace/BART
df_emails['summary_BART_sim'] = ''          # calcul de similarité par rapport à body_clean
df_emails['summary_T5'] = ''                # abstractive summary by HugginFace/T5
df_emails['summary_T5_sim'] = ''            # calcul de similarité par rapport à body_clean
df_emails['best_sim'] = ''                  # meilleure taux de similarité par rapport à body_clean
# df_emails.info()


In [ ]:
#
# A ce stade le fichier a la structure nécessaire à Melusine pour le nettoyage
#

# Fonction Data cleaning

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

def data_clean (tokens):
# Suppression des caractères spéciaux ou inutiles \"',]
#  print("Data cleaning --\n")
#  print("Avant :\n",type(tokens)," ",tokens)
   r = re.compile(r"[\\\''\'\"\[\]]+")
   tok=r.sub('', tokens)
   r = re.compile(r"([\(\)]{1,}|,{2,})")
   tok2=r.sub(' ', tok)
   r = re.compile(r"((\.\s?,)+|(, ,)|(^,{1,}))")
   tok3=r.sub('', tok2)
   r = re.compile(r"\s{2,}")
   texte=r.sub(' ', tok3)

#  texte = sent_tokenize(texte)  # découpe en phrases stockées dans une liste
#  print("Après sent_tokenize :\n",type(texte)," ",texte)
   return texte

# step 2
def data_clean_header_in_body(header,body):
  # Suppression de tout ce qui précède l'objet (inclus) dans le corps du mail
  # header = df_emails['header'][id]
  # body = df_emails['body_clean'][id]
  ref = 'Subject: '+str(header)+' '
  result = body.find(ref)
  if result != -1:
    result +=len(ref)
    texte = body[result:]
  else:
    texte = body

  return texte

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def remove_stop_words(text,stopwords):
  lst=[]
#  for token in str(text).split():
  for token in text.split():
    if token.lower() not in stopwords:    #checking whether the word is not 
      lst.append(token)                    #present in the stopword list.      
#    else:
#      print("suppression du stopword : ",token)
  # Join items in the list
#   print("Original text before removing stopwords  :   \n",text)
  result = ' '.join(lst)
#   print("Text after removing stopwords  :   \n",result)
  return result

In [ ]:
# Transforme Text en Dictionnaire - Key = indice de phrase - Value = Mots de la phrase
import re
from nltk.tokenize import sent_tokenize,word_tokenize

def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico



# Fonctions Summary methods

In [ ]:
import pandas as pd
import numpy as np

def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize_TFIDF(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['body_dict']
    artikle = corpus[i]
    texte=df['body'][i]
    
    if len(corpus) <= n_sentences:
      print("Longeur corpus inférieure au nombre minimal de phrases retenu pour le résumé")
      texte = "See Original"
      return texte
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
#    print('best sentences :',result)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    summary = [tokenized_article[i] for i in result]
    
    # transformation finale en chaine de caracteres
    texte = ''.join(summary)
    return texte

def summarize_spacy (text,ratio):
  # https://www.numpyninja.com/post/text-summarization-through-use-of-spacy-library
  # print('\n Texte à résumer avec un ratio de ',ratio*100,"%")
  # print(text)
  nlp = spacy.load('en_core_web_lg')
  doc= nlp(text)
  tokens=[token.text for token in doc]
#  print("Tokens : \n",tokens)

  # calcul frequence de mots
  word_frequencies={}
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
  # print("\n word_frequencies : ",word_frequencies)

  # normalisation des frequence de mots
  max_frequency=max(word_frequencies.values())
  # print('Max Frequency : ',max_frequency)
  for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency
  # print("\n Normalized word_frequencies : \n",word_frequencies)
  # Sentences token
  sentence_tokens= [sent for sent in doc.sents]
  # print("sentence_tokens ",sentence_tokens)
  # Calculate the most important sentences by adding the word frequencies in each sentence.
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():                            
          sentence_scores[sent]=word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent]+=word_frequencies[word.text.lower()]

  # identifier % (ratio) du texte avec score maximum
  from heapq import nlargest
  select_length=int(len(sentence_tokens)*ratio)
  # print('select_length ',select_length)
  if select_length != 0:
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
  else:
    summary = 'See Original'
  return summary



In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 13.2 MB/s 
     |████████████████████████████████| 212 kB 74.1 MB/s 
     |████████████████████████████████| 141 kB 67.0 MB/s 
     |████████████████████████████████| 127 kB 73.9 MB/s 
     |████████████████████████████████| 1.2 MB 45.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:

from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
my_model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def summarize_T5 (text):
  text = "summarize:" + text
  # encoding the input text
  input_ids=tokenizer.encode(text, return_tensors='pt', max_length=512)
  # Generating summary ids
  summary_ids = my_model.generate(input_ids)
  # Decoding the tensor and printing the summary.
  t5_summary = tokenizer.decode(summary_ids[0])
  return t5_summary
  



Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
# Loading the model and tokenizer for bart-large-cnn
tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

def summarize_BART (text):
  # Encoding the inputs and passing them to model.generate()
  inputs = tokenizer.batch_encode_plus([text],return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'], early_stopping=True)
  # Decoding and printing the summary
  bart_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  # print('bart_summary',bart_summary)
  return bart_summary


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

# Fonction - Sauvegarde en .csv

In [ ]:
# SAUVEGARDE en csv du résultat.
def svgd_csv_xls (file_path,file_name):

# to .CSV
  file_extension = ".csv"
  file = file_path+file_name+file_extension
  print("File : ",file)
  df_emails.to_csv(file, encoding='utf-8', index=False)
# df_emails[deb:fin].to_csv(file, encoding='utf-8', index=False)
  print("Svgde effectuée")
  return


# Boucle de traitement

Ci-dessous, on a séparé tous les traitements en blocs distincts pour pouvoir les re-executer séparément en cas de probleme.

In [ ]:
deb = 0
fin = df_emails.shape[0] # pb a résoudre sur un record en fin de fichier


In [ ]:
file_path = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/"
file_name = "enron_unique_output"
pfx = "_"+str(deb)
sfx = "_"+str(fin)

file_name = file_name+pfx+sfx

#svgd_csv_xls (file_path,file_name)

Data Cleaning

In [ ]:
from pandas._libs.parsers import TextReader
# Data Cleaning 1/2
#
print("DataCleaning en cours pour ",fin, "records.")
for id in range(deb,fin):
  print('ID:',id)
# step 1 : suppression de tous les caractères parasites
  df_emails['body_clean'][id] = data_clean(str(df_emails['body'][id]))
# step 2 : Suppression de tout ce qui précède l'objet (inclus) dans le corps du mail
  try:
    header = df_emails['header'][id]
    body = df_emails['body_clean'][id]
    df_emails['body_clean'][id] = data_clean_header_in_body(header,body)
  except:
    print('erreur retraitement body ID',id)
print("DataCleaning terminé pour ",fin, "records.")

DataCleaning en cours pour  500 records.
ID: 0
ID: 1
ID: 2
ID: 3
ID: 4
ID: 5
ID: 6
ID: 7
ID: 8
ID: 9
ID: 10
ID: 11
ID: 12
ID: 13
ID: 14
ID: 15
ID: 16
ID: 17
ID: 18
ID: 19
ID: 20
ID: 21
ID: 22
ID: 23
ID: 24
ID: 25
ID: 26
ID: 27
ID: 28
ID: 29
ID: 30
ID: 31
ID: 32
ID: 33
ID: 34
ID: 35
ID: 36
ID: 37
ID: 38
ID: 39
ID: 40
ID: 41
ID: 42
ID: 43
ID: 44
ID: 45
ID: 46
ID: 47
ID: 48
ID: 49
ID: 50
ID: 51
ID: 52
ID: 53
ID: 54
ID: 55
ID: 56
ID: 57
ID: 58
ID: 59
ID: 60
ID: 61
ID: 62
ID: 63
ID: 64
ID: 65
ID: 66
ID: 67
ID: 68
ID: 69
ID: 70
ID: 71
ID: 72
ID: 73
ID: 74
ID: 75
ID: 76
ID: 77
ID: 78
ID: 79
ID: 80
ID: 81
ID: 82
ID: 83
ID: 84
ID: 85
ID: 86
ID: 87
ID: 88
ID: 89
ID: 90
ID: 91
ID: 92
ID: 93
ID: 94
ID: 95
ID: 96
ID: 97
ID: 98
ID: 99
ID: 100
ID: 101
ID: 102
ID: 103
ID: 104
ID: 105
ID: 106
ID: 107
ID: 108
ID: 109
ID: 110
ID: 111
ID: 112
ID: 113
ID: 114
ID: 115
ID: 116
ID: 117
ID: 118
ID: 119
ID: 120
ID: 121
ID: 122
ID: 123
ID: 124
ID: 125
ID: 126
ID: 127
ID: 128
ID: 129
ID: 130
ID: 131
ID: 132
ID: 1

TF-IDF

In [ ]:

print("Traitement TF-IDF en cours pour ",fin, "records.")
for id in range(deb,fin):
  print(" id : ",id)
  text = df_emails['body_clean'][id]
  # Extractive summary using TF-IDF
  # ------------------------------------------------------------------------
  # suppression de stopwords pour TFIDF
  text = remove_stop_words(text,stopwords)

  # Dictionnarize the text to be used with TF-IDF
  df_emails['body_dict'][id] = dictionarize(text)   # sans stopwords
  max_sent = 3 
  if len(df_emails['body_dict'][id]) > max_sent:
    text = df_emails['body_dict'][id]
    # on affiche phrase par phrase le contenu de chaque mail
    # for cle, valeur in text.items():
    #   print("Ligne ", cle, " : ", valeur)
    # Identification des n_sent les plus significatives

    df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, max_sent, df_emails)
    ratio = len(df_emails['summary_TFIDF'][id])/len(df_emails['body_clean'][id])
    # print("\nRésumé TFIDF - Len ",len(df_emails['summary_TFIDF'][id])," Ratio : ",int(ratio*100),"%")
    # print(df_emails['summary_TFIDF'][id])
  else:
    # print("ID ",id," contenu trop court. Reprise de l'original pour TFIDF")
    df_emails['summary_TFIDF'][id] = df_emails['body_clean'][id]


print("Traitement TF-IDF terminé pour ",fin, "records.")
svgd_csv_xls (file_path,file_name)

Traitement TF-IDF en cours pour  500 records.
 id :  0
 id :  1
 id :  2
 id :  3
 id :  4
 id :  5
 id :  6
 id :  7
 id :  8
 id :  9
 id :  10
 id :  11
 id :  12
 id :  13
 id :  14
 id :  15
 id :  16
 id :  17
 id :  18
 id :  19
 id :  20
 id :  21
 id :  22
 id :  23
 id :  24
 id :  25
 id :  26
 id :  27
 id :  28
 id :  29
 id :  30
 id :  31
 id :  32
 id :  33
 id :  34
 id :  35
 id :  36
 id :  37
 id :  38
 id :  39
 id :  40
 id :  41
 id :  42
 id :  43
 id :  44
 id :  45
 id :  46
 id :  47
 id :  48
 id :  49
 id :  50
 id :  51
 id :  52
 id :  53
 id :  54
 id :  55
 id :  56
 id :  57
 id :  58
 id :  59
 id :  60
 id :  61
 id :  62
 id :  63
 id :  64
 id :  65
 id :  66
 id :  67
 id :  68
 id :  69
 id :  70
 id :  71
 id :  72
 id :  73
 id :  74
 id :  75
 id :  76
 id :  77
 id :  78
 id :  79
 id :  80
 id :  81
 id :  82
 id :  83
 id :  84
 id :  85
 id :  86
 id :  87
 id :  88
 id :  89
 id :  90
 id :  91
 id :  92
 id :  93
 id :  94
 id :  95
 id 

Spacy

In [ ]:

print("Résumé par Spacy en cours pour ",fin, "records.")
for id in range(deb,fin):
  print(" id : ",id)
# ------------------------------------------------------------------------
  # Extractive summary using Spacy
  ratio = 0.20
  text = df_emails['body_clean'].iloc[id]
  df_emails['summary_spacy'][id] = summarize_spacy (str(text),ratio)
  ratio = len(df_emails['summary_spacy'][id])/len(df_emails['body_clean'][id])
  # print("\nRésumé Spacy - Len ",len(df_emails['summary_spacy'][id]),"Ratio : ",int(ratio*100),"%")
  # print(df_emails['summary_spacy'][id])

print("Résumé par Spacy terminé pour ",fin, "records.")
svgd_csv_xls (file_path,file_name)

Résumé par Spacy en cours pour  500 records.
 id :  0
 id :  1
 id :  2
 id :  3
 id :  4
 id :  5
 id :  6
 id :  7
 id :  8
 id :  9
 id :  10
 id :  11
 id :  12
 id :  13
 id :  14
 id :  15
 id :  16
 id :  17
 id :  18
 id :  19
 id :  20
 id :  21
 id :  22
 id :  23
 id :  24
 id :  25
 id :  26
 id :  27
 id :  28
 id :  29
 id :  30
 id :  31
 id :  32
 id :  33
 id :  34
 id :  35
 id :  36
 id :  37
 id :  38
 id :  39
 id :  40
 id :  41
 id :  42
 id :  43
 id :  44
 id :  45
 id :  46
 id :  47
 id :  48
 id :  49
 id :  50
 id :  51
 id :  52
 id :  53
 id :  54
 id :  55
 id :  56
 id :  57
 id :  58
 id :  59
 id :  60
 id :  61
 id :  62
 id :  63
 id :  64
 id :  65
 id :  66
 id :  67
 id :  68
 id :  69
 id :  70
 id :  71
 id :  72
 id :  73
 id :  74
 id :  75
 id :  76
 id :  77
 id :  78
 id :  79
 id :  80
 id :  81
 id :  82
 id :  83
 id :  84
 id :  85
 id :  86
 id :  87
 id :  88
 id :  89
 id :  90
 id :  91
 id :  92
 id :  93
 id :  94
 id :  95
 id :

ValueError: ignored

T5

In [ ]:

print("Résumé par TF T5 en cours pour ",fin, "records.")
for id in range(deb,fin):
  print(" id : ",id)
# ------------------------------------------------------------------------
# Abstractive summary using Transformers T5
  # print("\nExtractive summary using Transformers / T5")
  text = df_emails['body_clean'].iloc[id]
  # print('text :',text)
  if len(df_emails['body_dict'].iloc[id]) !=0:
    try:
      df_emails['summary_T5'][id] = summarize_T5 (text)
      ratio = len(df_emails['summary_T5'][id])/len(df_emails['body_clean'][id])
      # print("\nRésumé Transformers T5  - Len ",len(df_emails['summary_T5'][id])," Ratio : ",int(ratio*100),"%")
    except:
      # print("Pb extractive summary on Id ",id)
      df_emails['summary_T5'][id] = "See Original"

print("Résumé par TF T5 terminé pour ",fin, "records.")
svgd_csv_xls (file_path,file_name)

In [ ]:
print("Traitement résume avec BART en cours pour ",fin, "records.")
for id in range(deb,fin):
  print("id : ",id)
  # print('--------------------')
  # coller quand OK pour les tests unitaires
  df_emails['body_clean'][id] = data_clean(str(df_emails['body'].iloc[id]))
  text = df_emails['body_clean'][id]
  # print("Cleaned text : ",len(str(text))," car.\n")
  # print(text)
 
# ------------------------------------------------------------------------
# Abstractive summary using Transformers BART
  # print("\nExtractive summary using Transformers / BART")
  text = df_emails['body_clean'].iloc[id]
  # print('text :',text)
  try:
    df_emails['summary_BART'][id] = summarize_BART (text)
    ratio = len(df_emails['summary_BART'][id])/len(df_emails['body_clean'][id])
    # print("\nRésumé Transformers BART  - Len ",len(df_emails['summary_BART'][id])," Ratio : ",int(ratio*100),"%")
  except:
    # print("Pb extractive summary on Id ",id)
    df_emails['summary_BART'][id] = "Erreur"

  # Svgd version intermédiaire
  if int(id/100) == id/100:
    file_name_tmp = "enron_unique_output_"+str(id)
    file = file_path+file_name_tmp
    svgd_csv_xls (file_path,file_name_tmp)
    print(file_name_tmp," sauvegardé.")

print("Traitement résume avec BART terminé pour ",fin, "records.")

svgd_csv_xls (file_path,file_name)

In [ ]:
# doc1.similarity(doc2)
#!python -m spacy download en_core_web_lg

# SPACY - Summary (Abstractive / Word Embedding)

In [ ]:
#!pip install spacy-transformers

In [ ]:
# https://spacy.io/usage/embeddings-transformers
# https://spacy.io/api/transformer


In [ ]:
# https://manusidtech.com/index.php/2021/08/14/complete-nlp-web-app-using-spacy-and-hugging-face-transformers/
#!pip install datasets transformers[sentencepiece]
#text = str(df_emails['body'][32])
#text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
#print(text)

#print(summarize_abstractive (text))
#print(summarize_spacy (text))
#print()

A voir

In [ ]:
# https://www.machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/
# https://towardsdatascience.com/abstractive-summarization-using-pytorch-f5063e67510
# https://www.analyticsvidhya.com/blog/2021/10/text-summarization-using-the-conventional-hugging-face-transformer-and-cosine-similarity/


In [ ]:
#!python -m spacy download en_core_web_lg

# Calcul similarités des résumés

In [ ]:
spacy.load('en_core_web_lg')

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
print("Traitement en cours pour ",fin, "records")

for id in range(deb,fin):
  print("ID:",id)
  # Calcul de similarité avec texte initial nettoyé
  doc1 = nlp(df_emails['body_clean'][id])
  max_sim = 0
  doc2 = nlp(df_emails['summary_TFIDF'][id])
  if doc2 != "":
    sim = doc1.similarity(doc2)
    df_emails['summary_TFIDF_sim'][id] = sim
    max_sim = sim

  doc2 = nlp(df_emails['summary_spacy'][id])
  if doc2 != "":
    sim = doc1.similarity(doc2)
    df_emails['summary_spacy_sim'][id] = sim
    if sim > max_sim:
      max_sim = sim

  doc2 = nlp(df_emails['summary_T5'][id])
  if doc2 != "Erreur":
    sim = doc1.similarity(doc2)
    df_emails['summary_T5_sim'][id] = sim
    if sim > max_sim:
      max_sim = sim

  doc2 = nlp(df_emails['summary_BART'][id])
  if doc2 != "Erreur":
    sim = doc1.similarity(doc2)
    df_emails['summary_BART_sim'][id] = sim
    if sim > max_sim:
      max_sim = sim

  # print("Meilleur score : ",max_sim)
  df_emails['best_sim'][id] = max_sim


svgd_csv_xls (file_path,file_name)

# NER with Spacy 1/2

In [ ]:
# https://www.thepythoncode.com/article/named-entity-recognition-using-transformers-and-spacy

import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')
spacy_stopwords = nlp.Defaults.stop_words
stopwords = spacy_stopwords
print("Stopwords :",spacy_stopwords)

for x in range(deb,fin):
# subject
  subj = df_emails.iloc[x]['header']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_header'][x] = doc_subj.ents

# body
  subj = df_emails.iloc[x]['body']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_body'][x] = doc_subj.ents

file_path = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/"
file_name = "enron_unique_output"
svgd_csv_xls (file_path,file_name)

# NER with Spacy 2/

In [ ]:
import sys
sys.exit()

In [ ]:
!pip install --upgrade transformers sentencepiece
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.2.0/en_core_web_trf-3.2.0-py3-none-any.whl
!python -m spacy download en_core_web_sm




In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd

file_path = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/"
file_name = "enron_unique_output.csv"
file = file_path+file_name

df_emails = pd.read_csv(file)
df_emails.tail(10)
deb = 0
fin = df_emails.shape[0]


In [ ]:
import spacy
def ner_sm(text):
  # load the English CPU-optimized pipeline
  # print('Modele : en_core_web_sm')
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)
  # spacy.displacy.render(doc, style="ent", jupyter=True)
  return doc.ents

def ner_trf(text):
  # load the English transformer pipeline (roberta-base) using spaCy
  # print('Modele : en_core_web_trf')
  nlp_trf = spacy.load('en_core_web_trf')
  # perform inference on the model
  doc_trf = nlp_trf(text)
  # display the doc with jupyter mode
  # spacy.displacy.render(doc_trf, style="ent", jupyter=True)
  return doc_trf


In [ ]:
print(deb,fin)
print(file_path)
print(file_name)

for id in range(deb,fin):
  print('ID:',id,)
  # subject
  text = str(df_emails.iloc[id]['header'])
  #print(id,' - header :',text)
  df_emails['NER_header'][id] = ner_sm(text)
#  print("df_emails['NER_header'][id] :",df_emails['NER_header'][id])

  # body
  text = str(df_emails.iloc[id]['body_clean'])
  #print(id,' - body :',text)
  df_emails['NER_body'][id] = ner_sm(text)
#  print("df_emails['NER_body'][id] :",df_emails['NER_body'][id])
#  print("\n")
#  print(df_emails['NER_header'][id])
#  print(df_emails['NER_body'][id])

  
  # Svgd version intermédiaire
  if int(id/100) == id/100:
    file_name_tmp = "enron_unique_output_"+str(id)+".csv"
    file = file_path+file_name_tmp
    svgd_csv_xls (file_path,file_name_tmp)
    print(file_name_tmp," sauvegardé.")   



In [ ]:
# Svgd version finale
file_name = "enron_unique_output_"+str(deb)+"_"+str(id)+".csv"
svgd_csv_xls (file_path,file_name)
print(file_name," sauvegardé (version finale).")

# Archived

In [ ]:
int(95/100) == 95/100
# rajouter svgd tous les 100 mails
# if id 
# decimal_part = id % 1

# Clusterisation des mails

In [ ]:
# pour les classifier par theme(s) en fonction des NER filtrés (en enlevant les dates par exemple)
# https://colab.research.google.com/drive/1BlK5Jxn7BQQwQhYhD-q6nGD9_wVbIYuq?usp=sharing#scrollTo=HdICm_4AyeOX

In [ ]:
# To Do ?
# identifier réponses et forward avec ou sans commentaires

In [ ]:
voir la suite 
https://colab.research.google.com/drive/1SM5ezLrJ4IOW86DoxEjbpULvqctD6mQC#scrollTo=1h6oY3hHbFFu&line=1&uniqifier=1